In [38]:
import os
from openai import OpenAI

# get your api key at https://app.nous.mesolitica.com/

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI(
    base_url="https://llm-router.nous.mesolitica.com",
)

In [11]:
# !wget https://huggingface.co/datasets/NousResearch/json-mode-eval/resolve/main/data/train-00000-of-00001.parquet

In [52]:
import pandas as pd
import json
import numpy as np

df = pd.read_parquet('train-00000-of-00001.parquet')
df.head()

,prompt,completion,schema
0,[{'content': 'You are a helpful assistant that...,"{""ssid"": ""OfficeNetSecure"", ""securityProtocol""...","{""title"": ""WirelessAccessPoint"", ""type"": ""obje..."
1,[{'content': 'You are a helpful assistant that...,"{""/"": {""device"": ""/dev/sda1"", ""mount_point"": ""...","{""$id"": ""https://example.com/fstab"", ""$schema""..."
2,[{'content': 'You are a helpful assistant that...,"{""campaignID"": ""CAMP123456"", ""productID"": ""PRO...","{""title"": ""PromotionalCampaign"", ""type"": ""obje..."
3,[{'content': 'You are a helpful assistant that...,"{""reservationID"": ""AH-158394"", ""guestName"": ""A...","{""title"": ""RestaurantReservation"", ""type"": ""ob..."
4,[{'content': 'You are a helpful assistant that...,"{""HomeImprovement"": {""room_interest"": ""living ...","{""type"": ""object"", ""properties"": {""HomeImprove..."


In [57]:
row = df.iloc[1].to_dict()
row

{'prompt': array([{'content': "You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{'$id': 'https://example.com/fstab', '$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'required': ['/'], 'properties': {'/': {'type': 'object', 'properties': {'device': {'type': 'string'}, 'mount_point': {'type': 'string'}, 'file_system_type': {'type': 'string'}, 'options': {'type': 'string'}, 'dump': {'type': 'string', 'pattern': '^[0-1]$'}, 'pass': {'type': 'string', 'pattern': '^[0-2]$'}}, 'required': ['device', 'mount_point', 'file_system_type', 'options', 'dump', 'pass']}}, 'patternProperties': {'^(/[^/]+)+$': {'type': 'object', 'properties': {'device': {'type': 'string'}, 'mount_point': {'type': 'string'}, 'file_system_type': {'type': 'string'}, 'options': {'type': 'string'}, 'dump': {'type': 'string', 'pattern': '^[0-1]$'}, 'pass': {'type': 'string', 'pattern': '^[0-2]$'}}, 'required': ['device', 'mount_point', 'file_s

In [30]:
response = client.chat.completions.create(
    model="mallam-small",
    temperature=0.1,
    max_tokens=1024,
    messages=[
    {
        "role": "user",
        "content": row['prompt'][1]['content']
    }
],
    tools=[row['schema']]
)

In [47]:
set(list(json.loads(response.choices[0].message.content).keys())) & set(list(json.loads(row['completion']).keys()))

{'bandwidth', 'securityProtocol', 'ssid'}

In [58]:
from tqdm import tqdm

correct, results = [], []
for i in tqdm(range(len(df))):
    row = df.iloc[i].to_dict()
    response = client.chat.completions.create(
        model="mallam-small",
        temperature=0.1,
        max_tokens=1024,
        messages=[
        {
            "role": "user",
            "content": row['prompt'][1]['content']
        }
    ],
        tools=[row['schema']]
    )
    left = set(list(json.loads(response.choices[0].message.content).keys()))
    right = set(list(json.loads(row['completion']).keys()))
    correct.append(len(left & right) == len(right))
    results.append((left, right))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:55<00:00,  3.56s/it]


In [60]:
np.mean(correct)

0.65